In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime as dt
import calendar
import xlsxwriter
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [13]:
wd_path = os.getcwd()
file_path = wd_path + '\Outcomes targets.xlsx'
xl = pd.ExcelFile(file_path)
sheets = xl.sheet_names
print(sheets)

x = pd.read_excel(file_path, sheet_name = sheets[0])
providers = x['Service Name'].unique()

['ABP Targets', 'Contract Targets', 'Minimum Service Levels', 'Investor Base Case', 'Low Case', 'Medium Case', 'High Case', 'ES Number']


In [14]:
def parse_targets_monthly(service_name):
    target_dict = {}
    for x in sheets:
        if x in ['ABP Targets', 'Contract Targets', 'Minimum Service Levels', 'Investor Base Case']:
            df = pd.read_excel(file_path, sheet_name = x)
            df = df[df['Service Name'] == service_name].drop(columns = ['Service Name'])
            df.set_index('Outcome', inplace=True)
            df = df.T
            df.reset_index(drop=False, inplace=True)
            df.rename(columns = {'index': 'Date'}, inplace=True)
            df['Date'] = pd.to_datetime(df['Date'])
            df = df.append(df.copy().append(df.copy()))
            df.sort_values('Date', ascending = True, inplace=True)
            df.reset_index(drop=True, inplace=True)
            df.rename(columns = {'Date':'Quarter Start'}, inplace=True)
            df['Date'] = df['Quarter Start']
            df.reset_index(drop=True, inplace=True)
            
            for i in range(0, len(df)):
                if i == 0:
                    pass
                else:
                    df.loc[i, 'Date'] = df.loc[i-1, 'Date'] + pd.DateOffset(months = 1)
                    
            for c in df.columns:
                if c in ['Quarter Start', 'Date']:
                    pass
                else:
                    df[c] = round(df[c]/3,3)
            
            df.drop(columns=['Quarter Start'], inplace=True)
            df['Job Sustainments'] = df['Job Sustainments <16 hours'] + df['Job Sustainments > 16 hours']
            cols = df.columns.tolist()
            cols = cols[:3] + cols[-1:] + cols[3:-1]
            df = df[cols]
            if x == 'Investor Base Case':
                df.set_index('Date', inplace=True)
                df = df.T
                target_dict[x + ' mnthly'] = df
            else:
                df = df[np.logical_and(df['Date'] >= '2021-04-01', df['Date']< '2022-04-01')]
                df.set_index('Date', inplace=True)
                df = df.T
                target_dict[x + ' mnthly'] = df
        elif x == 'ES Number':
            df = pd.read_excel(file_path, sheet_name =x)
            target_dict[x] = df
            
    return target_dict

In [15]:
def parse_targets_annual(service_name):
    target_dict = {}
    for x in sheets:
        if x in ['ABP Targets', 'Contract Targets', 'Minimum Service Levels', 'Investor Base Case']:
            df = pd.read_excel(file_path, sheet_name = x)
            df = df[df['Service Name'] == service_name].drop(columns = ['Service Name'])
            df.set_index('Outcome', inplace=True)
            df = df.T
            df.reset_index(drop=False, inplace=True)
            df.rename(columns = {'index': 'Date'}, inplace=True)
            df['Date'] = pd.to_datetime(df['Date'])
#             df = df.append(df.copy().append(df.copy()))
#             df.sort_values('Date', ascending = True, inplace=True)
#             df.reset_index(drop=True, inplace=True)
#             df.rename(columns = {'Date':'Quarter Start'}, inplace=True)
#             df['Date'] = df['Quarter Start']
#             df.reset_index(drop=True, inplace=True)
            target_dict[x] = df
        elif x == 'ES Number':
            target_dict[x] = pd.read_excel(file_path, sheet_name = x)
    
    for k, v in target_dict.items():
        if k == 'ES Number':
            pass
        else:
    #         if k == 'Contract Targets':
            first_year = v.loc[0, 'Date'].year
            first_year_start = (str(first_year) + '-04-01')
            first_year_end = str(first_year + 1) + '-04-01'
            final_year = v.loc[v.index[-1], 'Date'].year
            eofy_index = v[v['Date'] == first_year_end].index[0]
            # [camden['Contract Targets']['Date'] == first_year_end]
            v['Year'] = np.where(v['Date'] < first_year_end, 1, 0)

            for i in range(eofy_index, len(v)):
                if i >= eofy_index and i < eofy_index + 4:
                    v.loc[i, 'Year'] = 2
                elif i >= eofy_index + 4 and i < eofy_index + 8:
                    v.loc[i, 'Year'] = 3
                elif i >= eofy_index + 8 and i < eofy_index + 12:
                    v.loc[i, 'Year'] = 4
                elif i >= eofy_index + 12 and i < eofy_index + 16:
                    v.loc[i, 'Year'] = 5
                elif i >= eofy_index + 16 and i < eofy_index + 20:
                    v.loc[i, 'Year'] = 6

            v = v.groupby('Year').sum()
            for i in range(1, len(v)+1):
                if i == 1:
                    v.loc[i, 'Date'] = pd.to_datetime(first_year_start)
                else:
                    v.loc[i, 'Date'] = v.loc[i-1, 'Date'] + pd.offsets.DateOffset(years=1)

            v['Job Sustainments'] = v['Job Sustainments <16 hours'] + v['Job Sustainments > 16 hours']
            cols = v.columns.tolist()
            cols = cols[:3] + cols[-1:] + cols[3:-1]
            v = v[cols]
            v = v.reset_index(drop=False)
            v = v.set_index(['Date', 'Year'])
            v = v.T
            v['Total'] = v.sum(axis=1)
            cols = v.columns.tolist()
            cols = cols[-1:] + cols[:-1]
            v = v[cols]
            target_dict[k] = v
        
#         else:
#             v = v[np.logical_and(v['Date'] >= '2021-04-01', v['Date'] <= '2022-01-01')]
#             v.set_index('Date', inplace=True)
#             v['Job Sustainments'] = v['Job Sustainments <16 hours'] + v['Job Sustainments > 16 hours']
#             cols = v.columns.tolist()
#             cols = cols[:3] + cols[-1:] + cols[3:-1]
#             v = v[cols]
#             v = v.T
#             v['Total'] = v.sum(axis=1)
#             cols = v.columns.tolist()
#             cols = cols[-1:] + cols[:-1]
#             v = v[cols]
#             target_dict[k] = v
    
#     for k, v in target_dict.items():
#         for i in range(1, len(v)+1):
#             if i == 1:
#                 v.loc[i, 'Date'] = pd.to_datetime(first_year_start)
#             else:
#                 v.loc[i, 'Date'] = v.loc[i-1, 'Date'] + pd.offsets.DateOffset(years=1)
        
#         v = v.reset_index(drop=False)
#         v = v.set_index(['Date', 'Year'])
#         v = v.T
#         v['Total'] = v.sum(axis=1)
#         cols = v.columns.tolist()
#         cols = cols[-1:] + cols[:-1]
#         v = v[cols]
#         target_dict[k] = v
        
    return target_dict

In [16]:
def dict_to_excel(target_dict, area_name):
    
    writer = pd.ExcelWriter('{} Targets.xlsx'.format(area_name), engine='xlsxwriter')
    wb = xlsxwriter.Workbook('{} Targets.xlsx'.format(area_name))
    for k, v in target_dict.items():
        ws = wb.add_worksheet(k) 
        v.to_excel(writer, sheet_name = k, index=True)

    writer.save()
    return writer

In [17]:
def main(service_name):
    dict_a = parse_targets_annual(service_name)
    dict_b = parse_targets_monthly(service_name)
    dict_c = {**dict_a, **dict_b}
    writer = dict_to_excel(dict_c,service_name)
    return writer

In [18]:
for x in providers:
    print(x)
    y = main(x)

Hillside Clubhouse
Twining Enterprise
Westminster Drug Project
Working Well Trust (Enfield)
Working Well Trust (Tower Hamlets)
Enable
Tower Project JET
